In [1]:
print('Enter 1 to search for multiple keywords')
print('Enter 2 to search by a single hashtag')
s = -1
n = int(input())
if n==1:
    print('Enter keywords separated by space')
    keywords = input()
    s = 1
if n==2:
    print('Enter a hashtag')
    keywords = input()
    s = 2

Enter 1 to search for multiple keywords
Enter 2 to search by a single hashtag
1
Enter keywords separated by space
looking for developer recommendations


In [2]:
import os, random, sys, time
from datetime import datetime
from urllib.parse import urlparse
from selenium import webdriver 
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [3]:
browser = webdriver.Chrome(executable_path=r'./chromedriver.exe')

In [4]:
browser.get('https://www.linkedin.com/uas/login')

In [5]:
file = open('config.txt')
lines = file.readlines()
username = lines[0]
password = lines[1]

In [6]:
elementID = browser.find_element_by_id('username')
elementID.send_keys(username)

In [7]:
elementID = browser.find_element_by_id('password')
elementID.send_keys(password)

In [8]:
#link = 'https://www.linkedin.com/feed/hashtag/internships/'
#link = 'https://www.linkedin.com/search/results/content/?keywords=research%2C%20intern'
content = [i for i in keywords.strip().split(' ')]
if s == 1:
    link = 'https://www.linkedin.com/search/results/content/?keywords='
    for jj in range(len(content)):
        if jj != len(content)-1:
            link = link + content[jj] + '%20'
        if jj == len(content)-1:
            link = link + content[jj]
if s == 2:
    link = 'https://www.linkedin.com/feed/hashtag/'
    link = link + content[0] + '/'

browser.get(link)

In [9]:
SCROLL_PAUSE_TIME = 3
# Get scroll height
last_height = browser.execute_script("return document.body.scrollHeight")
for i in range(10):
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(SCROLL_PAUSE_TIME)
    new_height = browser.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height
browser.find_element_by_tag_name('body').send_keys(Keys.CONTROL + Keys.HOME)

In [10]:
browser.find_element_by_tag_name('body').send_keys(Keys.CONTROL + Keys.HOME)
connect_buttons = browser.find_elements_by_class_name('feed-shared-inline-show-more-text__see-more-less-toggle')
for connect_button in connect_buttons:
    #WebDriverWait(browser, 20).until(EC.element_to_be_clickable(connect_buttons)).click()
    connect_button.click()
    #print(connect_button)

In [11]:
timestamp = datetime.now()
src = browser.page_source
soup = BeautifulSoup(src, 'lxml')

In [12]:
master_data = soup.find_all('div', {'class': 'feed-shared-update-v2'})
info = soup.find_all('div', {'class':'feed-shared-actor__meta'})

In [13]:
no_pages=2

In [14]:
a=[]
base_link=browser.current_url
for j in range(1,no_pages+1):
    if(j>1):
        curr_link=base_link+"&page="+str(j)
        browser.get(curr_link)
    for i in range(1,11):
        path="//div[@id='search-marvel-srp-scroll-container']/div[1]/div[1]/ul[1]/li["+str(i)+"]/div[1]/div[1]"
        browser.find_element_by_xpath(path).click()
        src = browser.page_source
        soup = BeautifulSoup(src, 'lxml')
        master_data = soup.find_all('div', {'class': 'feed-shared-update-v2'})
        info = soup.find_all('div', {'class':'feed-shared-actor__meta'})
        try:
            post_obj={}
            Name = master_data[0].find_all('div', {'class': 'feed-shared-actor__meta relative'})[0].find('span', {'dir':'ltr'}).get_text().strip()
            post_obj['Name']=Name
            Post_Designation = master_data[0].find_all('span', {'class': 'feed-shared-actor__description'})[0].get_text().strip()
            Content = master_data[0].find_all('div', {'class': 'feed-shared-update-v2__description-wrapper'})[0].get_text().strip()
            post_obj['Content']=Content
            #print(Post)
            post_obj['Post_Designation']=Post_Designation
            #print(Content)
            Profile_URL=master_data[0].find('div', {'class': 'feed-shared-actor display-flex feed-shared-actor--with-control-menu'}).find('a', {'class':'app-aware-link feed-shared-actor__container-link relative display-flex flex-grow-1'})
            Profile_URL=str(Profile_URL)
            start_index=179
            end_index=Profile_URL.find('"',start_index)
            Profile_URL=Profile_URL[start_index:end_index]
            post_obj['Profile_URL']=Profile_URL
            a.append(post_obj)
        except:
            pass
print('$$$$$$$$$$$$$$$$$$$$$$$$$*******************************************************************$$$$$$$$$$$$$$$$$$$$$$$$$$$$$')

$$$$$$$$$$$$$$$$$$$$$$$$$*******************************************************************$$$$$$$$$$$$$$$$$$$$$$$$$$$$$


In [15]:
a

[{'Name': 'Darshan Chhajed',
  'Content': 'I am looking for recommendations for professionals who can help me with Website - Web Application Design & Development.#WebDevelopment #findapro #website #uxdesigner #applicationdeveloper #mumbai #seo #digitalmarketing',
  'Post_Designation': 'Imports & Operations Manager at IG International Pvt Ltd.',
  'Profile_URL': 'https://www.linkedin.com/in/darshan-chhajed-465736142?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAACKi0g0BxZAO8vztsDp8Nk5MaPgK2R9sfPM'},
 {'Name': 'Mahyar Geramizadegan',
  'Content': 'I am looking for recommendations for professionals who can help me with Application Development #ApplicationDevelopment #findapro',
  'Post_Designation': 'Organization Manager at HRDerak',
  'Profile_URL': 'https://www.linkedin.com/in/mahyar-geramizadegan-56b296217?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAADauTKMBTTaTB3qkcHQlFVzmOu4o4X5ERJg'},
 {'Name': 'Abhay Saxena',
  'Content': 'I am looking for recommendations for professionals who 

In [16]:
df = pd.DataFrame(a)
df.index = np.arange(1, len(df)+1)
df.to_excel ('Linked_Data_Leads.xlsx', sheet_name = 'sheet1')